In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import os
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import ipysheet
import colorsys
import xlsxwriter
import csv
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
import matplotlib.pyplot as plt
import sys
import itertools
import time

In [2]:
# import python CWL files

module_path = os.path.abspath(os.path.join('/SSB'))
if module_path not in sys.path:
    sys.path.append(module_path)

from od_matrices import run, collect_groups, to_groups, plot_od_pixmap, compute_matrix_norms,collect_matrix_entries, generate_hist_plot, generate_bar_plot

module_path = os.path.abspath(os.path.join('../../python/python_src_preprocessing/target_learning'))
if module_path not in sys.path:
    sys.path.append(module_path)

from preprocessing.preprocessing import add_pedestrian_to_field
import plotting.densitymap as densitymap


# To-Do's

- ~~generate new mapping file~~
- load all pedestrian data (with new mapping)
- divide into 10 second groups
- sort out groups with ped count < 50
- add time sequences option for denstiy maps
- Generate density maps 


## Generate new mapping 

New mapping will move all combinations to -1,-1 which have a smaller pedestrian count than 0.001% of the total day. Combinations to be removed are in the file removed_mappings.csv

In [35]:
day = "20190905"
base = '/media/veracrypt1/objectstream_{0}/'.format(day)
# 6 7 8 9 10 11 12 13 14 15 16 17 18 19
new_dir = "csv{0}".format(day)
if new_dir not in os.listdir(): 
    os.makedirs(new_dir)
    
    
mapping = pd.read_csv(os.path.join(base, 'mapping_removed.csv'), dtype={'pedestrianId': 'str'})

# concaternate all selected hours of one day
od_complete_day = []
com_size = 0
hours = list(range(5,6))

In [47]:
minx = -47000;
maxx = 7000;
width = abs(minx)+abs(maxx)
miny = -9700;
maxy = 8900;
height = abs(miny) + abs(maxy)

number_of_groups_used = 0

full = 0
missing = 0
pedestian_count = []

test = 0

for h in hours:
    tick0 = time.time()
    frame = pd.read_csv(os.path.join(base, 'hours', 'hour-{0}.csv'.format(h)), dtype={'pedestrianId': 'str'})
    print(frame.head())
    frame.x = frame.x.add(width) # 56000, 18000
    frame.x = frame.x.div(40)
    frame.x = frame.x.round(0)
    frame.y = frame.y.add(height)
    frame.y = frame.y.div(40)
    frame.y = frame.y.round(0)
    print(frame.x.max(), frame.y.max())
    print(frame.head())
    
    od_complete_day.append(frame)
    com_size += frame.size
    # (freq, interval, skip_groups, off_set)
    parameters = ('S', 10, 1, 0)
    # divide pedestrians into 10s intervals, generate od-matrix
    groups, od_matrices, origins_str, destinations_str = collect_groups(frame, mapping, parameters, write2csv=False, prefix='{0}d-{1}h'.format(day,h))
    tick1 = time.time()
    print("Step 1 : ", tick1-tick0)
    tick0 = time.time()
    
    for group in groups: # 10 second intervals
        ped_count = len(group.pedestrianId.unique())
        pedestian_count.append(ped_count)
        #if ped_count >= 50:
        number_of_groups_used+=1
        start_g = group.datetime.iloc[0]
        end_g = group.datetime.iloc[-1]
        sub_groups = to_groups(group, 1, 'S', 1, 0)
        
        info = []
        for sub_group in sub_groups: # 1 second intervals in 10 second intervals
            try:
                start_sg = sub_group.datetime.iloc[0]
                end_sg = sub_group.datetime.iloc[-1]
                info.append('{:%d, %b %Y}_{:%I:%M:%S %p}'.format(start_sg,start_sg))
                
                resolution = 1
                gauss_bounds = 1
                sigma = 0.7
                pedestrian_radius = 0.195
                
                sub_group = sub_group.sort_values(by=['pedestrianId','timestamp'])
                pedestrians = sub_group.groupby(['pedestrianId'])
                
                pedestrians = list(map(lambda x: x[1], list(pedestrians)))
                if test == 0:
                    print(pedestrians.head(100))
                    #for ped in pedestrians:
                    #    plt.plot(ped[:,3],ped[:,4])
                    
                   # plt.show()
                        
                    #print(pedestrians.head())
                    test += 1
                
            except IndexError:
                info.append('-1')
                
        if '-1' in info or ped_count < 100:
            missing+=1
        else:
            full += 1
      
    tick1 = time.time()
    print("Step 2 :",tick1-tick0)
    print(full, missing)
    
plt.plot(list(range(0,360)),pedestian_count)
plt.show()
    # sort every group by pedestrian id, add every pedestrian within one group to the density field
    #add_pedestrian_to_field(ped, matrix, area, dimension, resolution, gauss_density_bound, sigma, ped_radius, general_density_matrix):
    


       timestamp pedestrianId        x        y
0  1567659600031     12243949 -36184.2  6200.46
1  1567659600031     12244407 -33265.6  6527.29
2  1567659600031     12245649 -29597.1 -3060.87
3  1567659600031     12245926 -26228.2  5504.83
4  1567659600031     12246554 -15069.6 -6930.34
1519.0 686.0
       timestamp pedestrianId      x      y
0  1567659600031     12243949  445.0  620.0
1  1567659600031     12244407  518.0  628.0
2  1567659600031     12245649  610.0  388.0
3  1567659600031     12245926  694.0  603.0
4  1567659600031     12246554  973.0  292.0
Step 1 :  31.681561946868896


AttributeError: 'list' object has no attribute 'head'

In [ ]:
# check that correct combinations where removed
#plt.rcParams['figure.figsize'] = [7, 7]
#plot_od_pixmap(od_matrix[0][2].astype(int), fig_title="Complete OD-Matrix {0} day".format(day), val_precision=0)

In [ ]:
# ped, matrix, area, dimension, resolution, gauss_density_bound, sigma, ped_radius, general_density_matrix)
    